In [1]:
import tensorflow as tf
from tensorflow.contrib.framework.python.ops.variables import get_or_create_global_step
from tensorflow.python.platform import tf_logging as logging
import inception_preprocessing
from inception_resnet_v2 import inception_resnet_v2, inception_resnet_v2_arg_scope
import os
import time
import scipy.io
from glob import glob
from random import shuffle

import numpy as np 
from dataset import generate_cats_and_dogs
from scipy.misc import imread

slim = tf.contrib.slim

## Download  and divide the dataset

In [2]:
generate_cats_and_dogs()

It seems the dataset has already been generated (remove 'train' and 'test' folders to generate it again)


In [3]:
filename_queue = tf.train.string_input_producer(tf.train.match_filenames_once("train/*/*.jpg"))

image_reader = tf.WholeFileReader()
key, image_file = image_reader.read(filename_queue)
S = tf.string_split([key],'/')
length = tf.cast(S.dense_shape[1],tf.int32)
# adjust constant value corresponding to your paths if you face issues. It should work for above format.
label = S.values[length - tf.constant(2, dtype=tf.int32)]
label = tf.string_to_number(label,out_type=tf.int32)
image = tf.image.decode_image(image_file, channels=3) 

image_size = 299
height, width, is_training = (image_size, image_size, True)

#Perform the correct preprocessing for this image depending if it is training or evaluating
image = inception_preprocessing.preprocess_image(image, height, width, is_training)

with tf.Session() as sess:
    # Required to get the filename matching to run.
    tf.global_variables_initializer().run()

    # Coordinate the loading of image files.
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    for i in range(20):
        # Get an image tensor and print its value.
        key_val, label_val, image_tensor = sess.run([key,label,image])
        print(image_tensor.shape)
        print(label_val)
        print(key_val)


    # Finish off the filename queue coordinator.
    coord.request_stop()
    coord.join(threads)

INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.FailedPreconditionError'>, Attempting to use uninitialized value matching_filenames
	 [[Node: matching_filenames/read = Identity[T=DT_STRING, _class=["loc:@matching_filenames"], _device="/job:localhost/replica:0/task:0/cpu:0"](matching_filenames)]]


OutOfRangeError: FIFOQueue '_0_input_producer' is closed and has insufficient elements (requested 1, current size 0)
	 [[Node: ReaderReadV2 = ReaderReadV2[_device="/job:localhost/replica:0/task:0/cpu:0"](WholeFileReaderV2, input_producer)]]

Caused by op 'ReaderReadV2', defined at:
  File "/Users/Vilstrup/anaconda/envs/py3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/Vilstrup/anaconda/envs/py3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/Vilstrup/anaconda/envs/py3/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/Vilstrup/anaconda/envs/py3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/Vilstrup/anaconda/envs/py3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/Users/Vilstrup/anacon

## Download the inception model

In [5]:
model_path ='model'
checkpoint_name = 'inception_resnet_v2.ckpt'

if not os.path.isdir(model_path):
    os.mkdir(model_path)
    !curl -O 'http://download.tensorflow.org/models/inception_resnet_v2_2016_08_30.tar.gz'
    !tar -xf 'inception_resnet_v2_2016_08_30.tar.gz'
    os.rename('inception_resnet_v2_2016_08_30.ckpt', '{}/{}'.format(model_path, checkpoint_name))
    os.remove('inception_resnet_v2_2016_08_30.tar.gz')

In [ ]:
#State where your checkpoint file is
checkpoint_file = '{}/{}'.format(model_path, checkpoint_name)

#State where your log file is at. If it doesn't exist, create it.
log_dir = './log'
if not os.path.isdir(log_dir):
    os.mkdir(log_dir)

#================= TRAINING INFORMATION ==================
#State the number of epochs to train
num_epochs = 1

#State the number of classes to predict:
num_classes = 2
num_samples = len(glob('train/*/*.jpg'))

#Learning rate information and configuration (Up to you to experiment)
initial_learning_rate = 0.0002
learning_rate_decay_factor = 0.7
num_epochs_before_decay = 2

#State your batch size
batch_size = 8
image_size = 299
height=image_size 
width=image_size 
is_training=True
preprocess_treads=2
min_queue_size=256

#======================= TRAINING PROCESS =========================
#Now we start to construct the graph and build our model
with tf.Graph().as_default() as graph:
    tf.logging.set_verbosity(tf.logging.INFO) #Set the verbosity to INFO level

    # Make a queue of file names including all the JPEG images files in the relative
    # image directory.
    filename_queue = tf.train.string_input_producer(
        tf.train.match_filenames_once("{}/*/*.jpg".format('train')))


    # Read an entire image file which is required since they're JPEGs, if the images
    # are too large they could be split in advance to smaller files or use the Fixed
    # reader to split up the file.
    image_reader = tf.WholeFileReader()

    # Read a whole file from the queue, the first returned value in the tuple is the
    # filename which we are ignoring.
    file_name, image_file = image_reader.read(filename_queue)

    # Retrive the correct label from the 
    S = tf.string_split([file_name],'/')

    path_length = tf.cast(S.dense_shape[1],tf.int32)
    # Get the parent directory, this is the second last folder in the path
    label = S.values[path_length - tf.constant(2, dtype=tf.int32)]
    label = tf.string_to_number(label,out_type=tf.int32) # Convert the label into an int

    # Decode the image as a JPEG file, this will turn it into a Tensor which we can
    # then use in training.
    image = tf.image.decode_jpeg(image_file)

    #Perform the correct preprocessing for this image depending if it is training or evaluating
    image = inception_preprocessing.preprocess_image(image, height, width, is_training)

    # Take each image into a mini-batch
    images, labels = tf.train.shuffle_batch(
        [image, label],
        batch_size=batch_size,
        num_threads=preprocess_treads,
        capacity=min_queue_size + 3 * batch_size,
        min_after_dequeue=min_queue_size)
    
    #Know the number steps to take before decaying the learning rate and batches per epoch
    num_batches_per_epoch = int(num_samples / batch_size)
    num_steps_per_epoch = num_batches_per_epoch #Because one step is one batch processed
    decay_steps = int(num_epochs_before_decay * num_steps_per_epoch)

    #Create the model inference
    with slim.arg_scope(inception_resnet_v2_arg_scope()):
        logits, end_points = inception_resnet_v2(images, num_classes=num_classes, is_training = True)

    #Define the scopes that you want to exclude for restoration
    exclude = ['InceptionResnetV2/Logits', 'InceptionResnetV2/AuxLogits']
    variables_to_restore = slim.get_variables_to_restore(exclude = exclude)

    #Perform one-hot-encoding of the labels (Try one-hot-encoding within the load_batch function!)
    one_hot_labels = slim.one_hot_encoding(labels, num_classes)
    
    #Performs the equivalent to tf.nn.sparse_softmax_cross_entropy_with_logits but enhanced with checks
    loss = tf.losses.softmax_cross_entropy(onehot_labels = one_hot_labels, logits = logits)
    total_loss = tf.losses.get_total_loss()    #obtain the regularization losses as well

    #Create the global step for monitoring the learning_rate and training.
    global_step = get_or_create_global_step()

    #Define your exponentially decaying learning rate
    lr = tf.train.exponential_decay(
        learning_rate = initial_learning_rate,
        global_step = global_step,
        decay_steps = decay_steps,
        decay_rate = learning_rate_decay_factor,
        staircase = True)

    #Now we can define the optimizer that takes on the learning rate
    optimizer = tf.train.AdamOptimizer(learning_rate = lr)

    #Create the train_op.
    train_op = slim.learning.create_train_op(total_loss, optimizer)

    #State the metrics that you want to predict. We get a predictions that is not one_hot_encoded.
    predictions = tf.argmax(end_points['Predictions'], 1)
    probabilities = end_points['Predictions']
    accuracy, accuracy_update = tf.contrib.metrics.streaming_accuracy(predictions, labels)
    metrics_op = tf.group(accuracy_update, probabilities)


    #Now finally create all the summaries you need to monitor and group them into one summary op.
    tf.summary.scalar('losses/Total_Loss', total_loss)
    tf.summary.scalar('accuracy', accuracy)
    tf.summary.scalar('learning_rate', lr)
    my_summary_op = tf.summary.merge_all()

    #Now we need to create a training step function that runs both the train_op, metrics_op and updates the global_step concurrently.
    def train_step(sess, train_op, global_step):
        '''
        Simply runs a session for the three arguments provided and gives a logging on the time elapsed for each global step
        '''
        #Check the time for each sess run
        start_time = time.time()
        total_loss, global_step_count, _ = sess.run([train_op, global_step, metrics_op])
        time_elapsed = time.time() - start_time

        #Run the logging to print some results
        logging.info('global step %s: loss: %.4f (%.2f sec/step)', global_step_count, total_loss, time_elapsed)

        return total_loss, global_step_count

    #Now we create a saver function that actually restores the variables from a checkpoint file in a sess
    saver = tf.train.Saver(variables_to_restore)
    def restore_fn(sess):
        return saver.restore(sess, checkpoint_file)

    #Define your supervisor for running a managed session. Do not run the summary_op automatically or else it will consume too much memory
    sv = tf.train.Supervisor(logdir = log_dir, summary_op = None, init_fn = restore_fn)


    #Run the managed session
    with sv.managed_session() as sess:
        for step in range(num_steps_per_epoch * num_epochs):
        # for step in xrange(1):
            #At the start of every epoch, show the vital information:
            if step % num_batches_per_epoch == 0:
                logging.info('Epoch %s/%s', step/num_batches_per_epoch + 1, num_epochs)
                learning_rate_value, accuracy_value = sess.run([lr, accuracy])
                logging.info('Current Learning Rate: %s', learning_rate_value)
                logging.info('Current Streaming Accuracy: %s', accuracy_value)

                # optionally, print your logits and predictions for a sanity check that things are going fine.
                logits_value, probabilities_value, predictions_value, labels_value = sess.run([logits, probabilities, predictions, labels])
                print('logits: \n', logits_value)
                print('Probabilities: \n', probabilities_value)
                print('predictions: \n', predictions_value)
                print('Labels:\n:', labels_value)

            #Log the summaries every 10 step.
            if step % 10 == 0:
                loss, _ = train_step(sess, train_op, sv.global_step)
                summaries = sess.run(my_summary_op)
                sv.summary_computed(sess, summaries)

            #If not, simply run the training step
            else:
                loss, _ = train_step(sess, train_op, sv.global_step)

        #We log the final training loss and accuracy
        logging.info('Final Loss: %s', loss)
        logging.info('Final Accuracy: %s', sess.run(accuracy))

        #Once all the training has been done, save the log files and checkpoint model
        logging.info('Finished training! Saving model to disk now.')
        # saver.save(sess, "./flowers_model.ckpt")
        sv.saver.save(sess, sv.save_path, global_step = sv.global_step)